#  贪心算法 CTC decoder解码deepspeech

In [ ]:
import numpy as np
from collections import Counter

# 定义标签列表
labels = [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '-']

def ctc_decoder(predictions, blank_index):
    # 初始化解码结果列表
    decoded = []

    # 遍历每个时间步
    for t in range(predictions.shape[0]):
        # 获取当前时间步的概率分布
        prob_distribution = predictions[t]

        # 通过 argmax 找到概率最大的标签索引
        argmax_index = np.argmax(prob_distribution)

        # 如果标签索引不是 blank_index，则将其添加到解码结果列表中
        if argmax_index != blank_index:
            decoded.append(argmax_index)

        # 去除连续重复的标签
        if len(decoded) >= 2 and decoded[-1] == decoded[-2]:
            decoded.pop()

    return decoded

# 读取 DeepSpeech 特征文件
with open('../asserts/training_data-HDTF400/split_video_25fps_deepspeech/RD_Radio1_000_corrected_deepspeech.txt', 'r') as f:
    lines = f.readlines()

# 遍历每一行，并进行 CTC 解码
decoded_text = ''
for line in lines:
    # 将字符串转换为数字列表
    nums = [float(x) for x in line.strip().split()]

    # 将数字列表转换为概率分布
    prob_distribution = np.array(nums)

    # 进行 CTC 解码
    decoded_result = ctc_decoder(prob_distribution.reshape((-1, len(labels))), len(labels) - 1)

    # 将解码结果转换为文本
    text = ''.join([labels[i] for i in decoded_result])

    # 拼接解码结果
    decoded_text += text

# 打印拼接后的解码结果
print(decoded_text)


#### 用wav2vec挑选参考帧

In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC 
# from datasets import load_dataset
import torch
import soundfile as sf

# load model and processor
processor = Wav2Vec2Processor.from_pretrained("./wav2vec2-xls-r-300m-timit-phoneme")
model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-xls-r-300m-timit-phoneme")

# Read and process the input
audio_input, sample_rate = sf.read("./template/英文tts.wav")
inputs = processor(audio_input, sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

# Decode id into string
predicted_ids = torch.argmax(logits, axis=-1)      
predicted_sentences = processor.batch_decode(predicted_ids)
print(predicted_sentences)


ModuleNotFoundError: No module named 'transformers'